In [14]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
#from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder,ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
import logging
llm=ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
               )
memory=ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=500,
    memory_key="history",
    return_messages=True,
)
examples=[
    {
        "영화":"파묘",
        "answer":"⚰️👻🔦",
    },{
        "영화": "나홀로집에",
        "answer": "🏠🎄😂"
    },{
        "영화": "덤앤더머",
        "answer": "🤣🚙🧥"
    },
]

example_prompt=ChatPromptTemplate.from_messages(
    [
        ("human","영화 {영화}에 대해 이미티콘 3개로 표현해줘"),
        ("ai","{answer}")
    ]
)
example_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)


prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human","{question}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain=RunnablePassthrough.assign(history=load_memory) | prompt | llm

logging.basicConfig(level=logging.INFO)
def get_movie_emoji(question):
    response = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": response.content})
    logging.info(f"{question} -> {response.content}\n")

# ✅ 7️⃣ 실행 테스트 (두 개의 영화 질문 후, 이전 질문 확인)
get_movie_emoji("인셉션")
get_movie_emoji("타이타닉")


💭🌌🔑

INFO:root:인셉션 -> 💭🌌🔑



🚢💔🌊

INFO:root:타이타닉 -> 🚢💔🌊



In [15]:
chain.invoke({"question":"내가 제일 처음 물어본 영화는?"})

당신이 처음 물어본 영화는 '인셉션'이었습니다.

AIMessageChunk(content="당신이 처음 물어본 영화는 '인셉션'이었습니다.")

In [16]:
chain.invoke({"question":"내가 마지막에 물어본 영화는?"})

마지막으로 물어보신 영화는 "덤앤더머"입니다.

AIMessageChunk(content='마지막으로 물어보신 영화는 "덤앤더머"입니다.')

In [17]:
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='인셉션'), AIMessage(content='💭🌌🔑'), HumanMessage(content='타이타닉'), AIMessage(content='🚢💔🌊')]}
